In [1]:
! pip install -q --upgrade langchain-google-genai chromadb PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 9.4 MB/s eta 0:00:00
  Installing build dependencies

In [2]:
! pip install -q --upgrade langchain-community langchain langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.2/806.2 kB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00


In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

import PyPDF2
from langchain.vectorstores import Chroma

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader, DirectoryLoader

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import Docx2txtLoader


from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

from langchain_google_genai import ChatGoogleGenerativeAI

In [4]:
import os

In [5]:
def urlLoad(url_path):

  loader = WebBaseLoader(url_path)
  documents = loader.load()

  # split the document into chunks
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 100)
  document_chunks = text_splitter.split_documents(documents)

  text_list = []
  source_list = []

  for i, document in enumerate(document_chunks):
    text_list.append(document.page_content)
    source_metadata = str(url_path)+str("_page_")+str(i+1)
    source_list.append(source_metadata)

  return text_list, source_list

In [6]:
def txtLoad(txt_path):

  loader = TextLoader(txt_path)
  documents = loader.load()

  # split the document into chunks
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 100)
  document_chunks = text_splitter.split_documents(documents)

  text_list = []
  source_list = []

  for i, document in enumerate(document_chunks):
    text_list.append(document.page_content)
    source_metadata = str(txt_path)+str("_page_")+str(i+1)
    source_list.append(source_metadata)

  return text_list, source_list

In [7]:
def pdfLoad(pdf_path):

  text_list = []
  source_list = []

  pdf_reader = PyPDF2.PdfReader(pdf_path)

  for i in range(len(pdf_reader.pages)):
    page_obj = pdf_reader.pages[i]
    text_data = page_obj.extract_text()
    page_obj.clear()

    text_list.append(text_data)
    source_list.append(str(pdf_path) + str("_page_") + str(i+1))

  return text_list, source_list

In [8]:
def docLoad(doc_path):

  loader = Docx2txtLoader(doc_path)
  documents = loader.load()

  # split the document into chunks
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 100)
  document_chunks = text_splitter.split_documents(documents)

  text_list = []
  source_list = []

  for i, document in enumerate(document_chunks):
    text_list.append(document.page_content)
    source_metadata = str(doc_path)+str("_page_")+str(i+1)
    source_list.append(source_metadata)

  return text_list, source_list

In [9]:
def returnChuncksFunction(url_file=None, txt_file=None, doc_file=None, pdf_file=None):

  text_main_list, source_main_list = [], []

  if url_file:
    url_text_file, url_source_file = urlLoad(url_file)
    for i,j in zip(url_text_file, url_source_file):
      text_main_list.append(i)
      source_main_list.append(j)

  elif txt_file:
    txt_text_file, txt_source_file = txtLoad(txt_file)
    for i,j in zip(txt_text_file, txt_source_file):
      text_main_list.append(i)
      source_main_list.append(j)

  elif doc_file:
    doc_text_file, doc_source_file = docLoad(doc_file)
    for i,j in zip(doc_text_file, doc_source_file):
      text_main_list.append(i)
      source_main_list.append(j)

  elif pdf_file:
    pdf_text_file, pdf_source_file = pdfLoad(pdf_file)
    for i,j in zip(pdf_text_file, pdf_source_file):
      text_main_list.append(i)
      source_main_list.append(j)

  # embeddings = OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY)
  # vector_store = Chroma.from_texts(text_main_list, embeddings, metadatas = [{"source" : s} for s in source_main_list])

  return text_main_list, source_main_list

In [11]:
# import google.generativeai as genai
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
# genai.configure(api_key=GOOGLE_API_KEY)

In [12]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size = 10000,
#                                                chunk_overlap = 1000)
# context = "\n\n".join(str(p.page_content) for p in pages)
# texts = text_splitter.split_text(context)

In [13]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001",
                                          google_api_key = GOOGLE_API_KEY)

In [20]:
persist_directory = "PersonalDB"

In [15]:
url_file = "https://en.wikipedia.org/wiki/Albert_Einstein"
txt_file = "/content/SampleTextFile_100kb.txt"
doc_file = "/content/file-sample_100kB.doc"
pdf_file = "/content/Conditional Generative Adversarial Nets.pdf"

text_main_list, source_main_list = returnChuncksFunction(url_file, txt_file, doc_file, pdf_file)

In [23]:
vector_index = Chroma.from_texts(texts = text_main_list,
                                 embedding = embeddings,
                                 metadatas = [{"source" : s} for s in source_main_list],
                                 persist_directory = persist_directory)

In [24]:
vector_index.persist()

In [25]:
vector_index

In [26]:
vector_index = None

In [27]:
vector_index

In [28]:
vector_index = Chroma(persist_directory = persist_directory,
                      embedding_function = embeddings)

In [29]:
vector_index

In [30]:
retriever = vector_index.as_retriever(search_kwargs={"k":5})

In [40]:
### Checking

query = "Tell me about Albert Einstein."
docs = retriever.get_relevant_documents(query)
docs

[Document(page_content='Retrieved from "https://en.wikipedia.org/w/index.php?title=Albert_Einstein&oldid=1211513439"', metadata={'source': 'https://en.wikipedia.org/wiki/Albert_Einstein_page_509'}),
 Document(page_content='Retrieved from "https://en.wikipedia.org/w/index.php?title=Albert_Einstein&oldid=1211513439"', metadata={'source': 'https://en.wikipedia.org/wiki/Albert_Einstein_page_509'}),
 Document(page_content="External links\n\n\nAlbert Einstein  at Wikipedia's sister projects\n\nMedia from CommonsQuotations from WikiquoteTexts from Wikisource\n\n\n\n\nScholia has an author profile for Albert Einstein.", metadata={'source': 'https://en.wikipedia.org/wiki/Albert_Einstein_page_473'}),
 Document(page_content="External links\n\n\nAlbert Einstein  at Wikipedia's sister projects\n\nMedia from CommonsQuotations from WikiquoteTexts from Wikisource\n\n\n\n\nScholia has an author profile for Albert Einstein.", metadata={'source': 'https://en.wikipedia.org/wiki/Albert_Einstein_page_473'})

In [41]:
model = ChatGoogleGenerativeAI(model = "gemini-pro",
                               google_api_key = GOOGLE_API_KEY,
                               temperature = 0.2,
                               convert_system_message_to_human = True)

In [42]:
qa_chain = RetrievalQA.from_chain_type(llm = model,
                                       chain_type = "stuff",
                                       retriever = retriever,
                                       return_source_documents = True
                                       )

In [43]:
response = qa_chain.invoke(query)
# response = qa_chain({"query": query})

In [44]:
response

{'query': 'Tell me about Albert Einstein.',
 'result': 'Albert Einstein was a German-born physicist who developed the theory of relativity, one of the two pillars of modern physics. His work is also known for its influence on the philosophy of science.',
 'source_documents': [Document(page_content='Retrieved from "https://en.wikipedia.org/w/index.php?title=Albert_Einstein&oldid=1211513439"', metadata={'source': 'https://en.wikipedia.org/wiki/Albert_Einstein_page_509'}),
  Document(page_content='Retrieved from "https://en.wikipedia.org/w/index.php?title=Albert_Einstein&oldid=1211513439"', metadata={'source': 'https://en.wikipedia.org/wiki/Albert_Einstein_page_509'}),
  Document(page_content="External links\n\n\nAlbert Einstein  at Wikipedia's sister projects\n\nMedia from CommonsQuotations from WikiquoteTexts from Wikisource\n\n\n\n\nScholia has an author profile for Albert Einstein.", metadata={'source': 'https://en.wikipedia.org/wiki/Albert_Einstein_page_473'}),
  Document(page_conten

In [45]:
response["result"]

'Albert Einstein was a German-born physicist who developed the theory of relativity, one of the two pillars of modern physics. His work is also known for its influence on the philosophy of science.'

In [46]:
response["source_documents"]

[Document(page_content='Retrieved from "https://en.wikipedia.org/w/index.php?title=Albert_Einstein&oldid=1211513439"', metadata={'source': 'https://en.wikipedia.org/wiki/Albert_Einstein_page_509'}),
 Document(page_content='Retrieved from "https://en.wikipedia.org/w/index.php?title=Albert_Einstein&oldid=1211513439"', metadata={'source': 'https://en.wikipedia.org/wiki/Albert_Einstein_page_509'}),
 Document(page_content="External links\n\n\nAlbert Einstein  at Wikipedia's sister projects\n\nMedia from CommonsQuotations from WikiquoteTexts from Wikisource\n\n\n\n\nScholia has an author profile for Albert Einstein.", metadata={'source': 'https://en.wikipedia.org/wiki/Albert_Einstein_page_473'}),
 Document(page_content="External links\n\n\nAlbert Einstein  at Wikipedia's sister projects\n\nMedia from CommonsQuotations from WikiquoteTexts from Wikisource\n\n\n\n\nScholia has an author profile for Albert Einstein.", metadata={'source': 'https://en.wikipedia.org/wiki/Albert_Einstein_page_473'})

In [ ]:
# vector_index = Chroma.from_texts(texts, embeddings).as_retriever(search_kwargs={"k":5})

In [ ]:
# question = "Describe the Multi-head attention layer in detail?"
# result = qa_chain({"query": question})
# result["result"]